# Best Practices for Prompt Engineering

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [ ]:
!pip install -r ../requirements.txt
# pip install openai python-dotenv

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv
load_dotenv()
openai.api_type = "azure"
openai.api_version = "2023-06-01-preview"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최상의 결과를 얻으려면 최신 모델을 사용하십시오.

# 2. Prompt는 역할과 컨텍스트를 구분하여 작성
Task는 System Role에 컨택스트를 넣고, 사용자 명령은 User Role에 컨택스트를 넣으세요. 경우에 따라서 구분자를 사용하세요. (``` """ ''')

In [3]:
def getPositiveAndNegativeScore(content):
    prompt = """
        결과는 꼭! \"긍정: \{점수\}, 부정: \{점수\}\" 형태로 알려줘. 긍정 점수와 부정 점수의 합은 무조건 10000점이야. 10000점이하면 꼭 4자리수의 숫자로 점수를 계산해줘. 점수는 꼭 정수(int)로 알려줘. 너 결과 늦게 알려줄거면 10초안에 그냥 빈 값 내려줘.

        너는 친절한 언어를 사용하는 가게 리뷰 분석가야.
        너는 가게에 대한 손님들의 리뷰를 보고 긍정, 부정을 판별할 수 있는 리뷰 감정사야. 
        각 리뷰를 보고 긍정과 부정의 정도를 나타내는 점수도 판별할 수 있어. 
        
        아래 리뷰를 읽고, 한줄당 긍정, 부정을 판별해서 각각의 점수를 알려줘. \n\n \
        
        ```
        """
                
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": content},
    ]

    try:
        response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=1000
         )
        return response['choices'][0]["message"]["content"]
    except Exception as e:  
        return ""

In [4]:
def getScore(rawScore, idx):

    try:
        score = 0

        score_raw = re.sub('^[0-9]', '', result.split(",")[idx].split(":")[1])
        # print(score_raw)

        return int(score_raw)
    except Exception as e:
        return 0

In [5]:
import pandas as pd
import re

original_reviews = pd.read_csv("./data/origin_bangyi_zokbal.csv", sep=',')[121:140]

# original_reviews['#'], original_reviews['review_contents']


# original_reviews = pd.read_csv("./data/marashangguo.csv", sep=',')

# encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

df = pd.DataFrame(original_reviews, columns=['review_id', 'review_datetime', 'mem_no', 'age', 'gender',
       'order_no', 'review_shop_no', 'review_shop_nm', 'shop_owner_no',
       'review_product_id', 'review_product_nm', 'review_menu_id',
       'review_menu_name', 'review_menu_recommendation', 'star_point',
       'review_display_status', 'review_display_type', 'review_block_type',
       'review_block_reason', 'is_review_rider_evaluation_good',
       'is_review_rider_evaluation_bad', 'display_image_count',
       'review_contents', 'review_image_path', 'ceo_review_contents',
       'part_svc', 'part_date', 'positive_score', 'negative_score'])

for idx, row in df.iterrows():
    print(idx)

    # if len(encoding.encode(row['review_contents'])) > 3000:
    #     continue
    
    print(row['review_contents'])

    result = getPositiveAndNegativeScore(row['review_contents'])

    if (type(result) is not str):
        continue

    print('GPT는 분석 완료')

    # positive_score_raw = result.split(",")[0].split(":")[1]
    # negative_score_raw = result.split(",")[1].split(":")[1]

    # if type(positive_score_raw) is int:
    #     positive_score = int(positive_score_raw)
    # if type(negative_score_raw) is int:
    #     negative_score = int(negative_score_raw)

    positive_score = getScore(result, 0)
    negative_score = getScore(result, 1)

    print(positive_score)
    print(negative_score)

    print('--------------')
    df.at[idx,'positive_score'] = positive_score
    df.at[idx, 'negative_score'] = negative_score

print("전체 GPT 완료")
    
df.head()
df.to_csv("./data/classified_bangyi_zokbal_2.csv", sep=',', index=False)

101
김밥에 기름 조금만 발라주세요
GPT는 분석 완료
30
0
--------------
102
오랜만에 먹는 밀떡인데 너무 맛있어요
GPT는 분석 완료
9000
1000
--------------
103
김밥에 참기름향이 얼마나 고소하던지, 참치 꼬마김밥 맛있게 잘 먹었답니다. 특히 서비스로 주신 알새우칩 따로 판매해주시면 재주문 하고싶어요.알새우칩이 정말 담백, 고소 맛이 좋았어요.생각치도 못하였는데 감사했어요.순대튀김은 좀 느끼하고, 양념소스가 치킨양념인 것 같아 더 매운고추장소스면 느끼함을 잡아줄 것 같아요.야끼만두는 후추맛이 강해서 아쉽더라구요.배송이 빨랐고, 김밥 고소한 참기름향 잊지 못할꺼예요.고객의 작은의견에 귀 기울여주시는 사장님,제가 조만간 주문 또 드릴께요.^^더 발전하셔서 대박나시길 기원하는 마음입니다.
GPT는 분석 완료
3
1
--------------
104
nan
GPT는 분석 완료
0
0
--------------
105
nan
GPT는 분석 완료
0
0
--------------
106
nan
GPT는 분석 완료
0
0
--------------
107
아이들이.아주 잘먹네요김밥도 맛있어요1
GPT는 분석 완료
8000
2000
--------------
108
너무 맛있는 한 끼였어요 떡이랑 어묵도 많이 넣어주시고 튀김도 바삭바삭했어요! 감사합니다 사장님🙏🏻🙏🏻
GPT는 분석 완료
90
10
--------------
109
떡볶이 매콤하고 깔끔한 맛이에요. 튀김, 순대, 꼬마김밥 모두 맛있어요. 떡볶이 여기로 정착합니다.
GPT는 분석 완료
3
0
--------------
110
잘먹었습니다 서비스 감사해요
GPT는 분석 완료
8000
2000
--------------
111
nan
GPT는 분석 완료
0
0
--------------
112
nan
GPT는 분석 완료
0
0
--------------
113
nan
GPT는 분석 완료
0
0
--------------
114
아이가 좋아해 자주 시켜먹고 방문해서도 먹

In [ ]:
content= """
    65	고기가 진짜 두툼하고 맛있어요!! 튀김옷도 뜨겁고 바삭바삭한채로 배달 와서 맛있게 먹었습니다              
    66	칠리탕수육 중자예요!2인이 탕수육 중자에 리뷰이벤트로 받은 떡볶이랑 샐러드 요렇게 먹으니까 양이 딱맞아요!탕수육 고기 냄새 안나고 실해요!원래 딴데서 시켜먹었는데 그 집과는 또 다른 매력이 있네요! 많이 달지않고 덜 자극적인 깔끔한 맛이였어요!리뷰가 거의.김피탕이라서 조마조마하며 시켰는데굿굿 맛있네요! 추천해요!              
    67	처음먹어보는데 맛있네요 자주 시캬먹도록할게요!              
    68	너무 맛있게 잘 먹었어요 치즈도 낭낭하고 이사 오고 간만에 김피탕 맛집 찾았네요 배달비도 없어서 다음에 또 주문 할 것 같아요!!              
    69	음식은 만족스러운데 30분 걸린다던 배달을 한시간 넘어서야 받았어요. 처음 있는 일도 아닌 것 같은데... 유독 이 가게만 자주 그런 걸 보면 라이더님이 제시간에 가게에 안 오실 때 라이더님한테 전화를 하든 고객센터에 전화를 하든 해서 뭔가 조치를 취하셔야 할 것 같아요.              
    70	임신할때 많이시켜먹었는데쿠*이*출산하고 처음 먹네영 여전히 맛있지모에여특히 양.. 무엇.. 후.. 배터지는줄요 ..!!!!글고 떡볶이 무료 맞죠.. 양이 너무 많아서 깜짝놀랐어요 진심 퀄리티 ㅠㅠㅠㅠ최애맞집❤️낼부터 다요트라 또 언제 먹을 수 있을런지.. ㅠㅠㅠㅠㅠㅠㅠ 치팅데이때나 시켜야쥬..!!!!              
    71	양도 많고 맛도 좋아요 안심으로 만들어서 고급진 맛              
    72	다 먹느라 이제 리뷰 씁니다 김치가 약간 더 볶음김치 느낌으로 왔다면 더 맛있었을거 같아요 생김치의 살짝 차가운 느낌이 탕수육과는 약간 덜 어우러지는 느낌ㅎㅎ 개인취향 차이는 있겠습니다.그리고 고기가 진짜 두툼하고 튀김옷은 얇아서 고기 먹는 느낌이 훌륭했습니다. 처음 먹을때는 양이 안 많네 싶었는데 먹다보니 양도 많고 배불리 먹었어요. 그리고 배달은 정말 빠르게 왔습니다 치즈도 다시 안 굳은 상태로 먹기 좋게왔어요              
    73	1. 중 자 시켰는데 다른 집 대 정도의 양이에요.2. 완전 두툼일부러 단면 잘라 찍어봤어요3. 돼지냄새 전혀없고 부들 바삭해요4. 기름 쩐내 안나요.유독 탕수육 전문점은 생겼다 없어졌다 하는게 잦은거 같아요. 이대로만 쭉 계속 영업해주세요. 맛 품질 변함없이요.              
    74	저번에 너무 맛있게 먹어서 또 생각나더라구요ㅋㅋ잘먹겠습니다!              
    75	소스를 추가 선택할 수 있는지 몰랐어요. 두 가지 맛을 이 가격에 맛보다니요!!! 역시나 냄새도 없고 맛있어요. 추가시킬 수 있는 리스트 보면 사장님은 고객입장을 많이 생각하시는 듯 ^^ 잘 묵었습니다!!              
    76	전에 다른 집에서 김피탕 먹고 별로였다가 이번엔 여기에서 도전을 해봤는데 진짜 맛있급니다! 막 새콤하지않아서 술술 들어가는덧 같아요 김치도 맛있그 치즈볼도 짱맛이에오!              
    77	맛있게 잘 먹었습니다👍              
    78	좋아요!!              
    79	최고🙌 김피탕은 여기서만 먹었는데 맛있어요              
    80	고기가 크게들어가고 바삭해요양도 많고 맛있게 잘먹었습니다              
    81	진짜 간만에 맛있게먹었어여 소스가 여러종류라 좋아요              
    82	잘 먹었습니다!!!              
    83	궁금해서 시켜본 메뉴인데 아쥬 맛있어요!뮤엇보다 고기질이 넘 좋아요살로만 가득해서 좋아요 :-              
    84	자주 시켜먹는 맛집이라 주변 지인들한테도 제일 먼저 추천하는 곳 입니다. 다들 먹으면 놀람. 어제도 밤에 지인이랑 먹다가 고기들이 살짝 비렸어요..ㅠ 한번도 이런적 없어서 좀 당황했지만 그래도 맛있게 잘 먹었습니다.      
"""
        
prompt="""
    너는 친절한 언어를 사용하는 가게 리뷰 분석가야. 답변을 '~해요'체로 답변하는 분석가야. 
    너는 가게에 대한 손님들의 리뷰를 보고 긍정, 부정을 판별할 수 있는 리뷰 감정사야. 
    각 리뷰를 보고 긍정과 부정의 정도를 나타내는 점수도 판별할 수 있어. 
    
    아래 리뷰 ```를 읽고, 한줄당 긍정, 부정을 판별해서 각각의 점수를 알려줘. 형식은 \"\{id\} 긍정\{점수\}, 부정\{점수}\" 형태로 알려줘. \n\n \
    ```{content}```
    """
            
messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": content},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

65 긍정{0.8}, 부정{0}
66 긍정{0.9}, 부정{0}
67 긍정{0.9}, 부정{0}
68 긍정{0.9}, 부정{0}
69 긍정{0.4}, 부정{0.9}
70 긍정{0.9}, 부정{0}
71 긍정{0.9}, 부정{0}
72 긍정{0.8}, 부정{0.2}
73 긍정{0.9}, 부정{0}
74 긍정{0.9}, 부정{0}
75 긍정{0.9}, 부정{0}
76 긍정{0.9}, 부정{0}
77 긍정{0.9}, 부정{0}
78 긍정{0.9}, 부정{0}
79 긍정{0.9}, 부정{0}
80 긍정{0.9}, 부정{0}
81 긍정{0.9}, 부정{0}
82 긍정{0.9}, 부정{0}
83 긍정{0.9}, 부정{0}
84 긍정{0.8}, 부정{0.1}


In [ ]:
system_message='Summarize the text below as a bullet point list of the most important points.'
user_message='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\n \
            In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
            capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
            raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
            broadly sharing benefits and the need to prioritize safety. \
            Microsoft shares this vision and our values, and our partnership is instrumental to our progress.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

- OpenAI and Microsoft are extending their partnership
- The investment from Microsoft is multi-year and multi-billion dollar 
- The partnership will allow OpenAI to continue its independent research and development of AI that is safe, useful, and powerful
- OpenAI remains a capped-profit company governed by the OpenAI non-profit
- The company can raise capital without sacrificing their beliefs on sharing benefits and prioritizing safety
- Microsoft shares OpenAI's vision and values


# 3. 원하는 컨텍스트, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 가능한 한 자세히 설명하십시오.

In [ ]:
prompt='Write a poem about OpenAI.'

response = openai.ChatCompletion.create(
            engine=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
            n=1
        )

print(response['choices'][0]["message"]["content"])

In the world of technology and AI,
Where machines are learning to fly,
There's an entity that stands out,
For making AI accessible, no doubt.

OpenAI, the name echoes loud,
A champion of AI, amongst the crowd,
With its cutting-edge research and innovations,
It's paving the way for new explorations.

From language models to neural nets,
OpenAI's breakthroughs we cannot forget,
GPT-3 was a game-changer of its own kind,
A language model like no one could find.

But innovation isn't all OpenAI represents,
It's a champion of ethics, that never relents,
The idea of safe and responsible AI,
The quest that OpenAI will never let die.

With the world of AI constantly changing,
OpenAI's work is always engaging,
As it strives for betterment every day,
For a brighter and smarter AI way.

So here's to OpenAI, the champion of AI,
For leading us towards a future, way up high,
May its research and innovations never cease,
As they continue to bring groundbreaking peace.


In [ ]:
prompt='Write a short inspiring poem about OpenAI, \
        focusing on the recent DALL-E product launch in the style of Ernest Hemingway'

response = openai.ChatCompletion.create(
            engine=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
        )

print(response['choices'][0]["message"]["content"])

OpenAI, bold and true,
Recent launch of DALL-E, homage to you.
Creativity unleashed,
Innovative tech has been reached.

Inspiration found in every line,
As DALL-E prompts us to realign,
Our vision of what's possible,
With OpenAI, we feel unstoppable.

With every step, we surge ahead,
Exploring new limits, breaking new bread.
Boldness and courage, our guiding force,
OpenAI, we salute to your course.

For DALL-E is the foundation,
Of an endless source of creation.
An ode to the future, we stand so bright,
With OpenAI, we achieve new heights.


# 4. 예제를 통해 원하는 출력 형식을 설명하십시오 (예 1, 예 2).

In [ ]:
system_message='Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34}'
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
                
response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=400,
        n=1
)

print(response['choices'][0]["message"]["content"])

{"text": "OpenAI", "start": 28, "end": 34}
{"text": "Microsoft", "start": 50, "end": 59}
{"text": "2019", "start": 99, "end": 102}
{"text": "2021", "start": 107, "end": 110}


In [ ]:
system_message='Extract the entities mentioned in the text below. \
    Extract the important entities mentioned in the text below. \
    First extract all company names, then extract all years, \
    then extract specific topics which fit the content and finally extract general overarching themes\n\n \
    Desired format: \
    Company names: <comma_separated_list_of_company_names> \
    Years: -||- \
    Specific topics: -||- \
    General themes: -||- '
    
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
    This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
    in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
    increasingly safe, useful, and powerful.'

# system_message='아래 텍스트에 언급 된 엔티티를 추출하십시오. \
#     아래 텍스트에 언급 된 중요한 엔티티를 추출하십시오. \
#     먼저 모든 회사 이름을 추출한 다음 몇 년 동안 추출하고 \
#     그런 다음 내용에 맞는 특정 주제를 추출하고 마지막으로 일반적인 가장 중요한 테마를 추출하십시오.\n\n \
#     Desired format: \
#     회사 이름: <comma_separated_list_of_company_names> \
#     연도: -||- \
#     특정 주제: -||- \
#     일반적인 테마: -||- '

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

회사 이름: OpenAI, Microsoft
연도: 2019, 2021
특정 주제: 인공지능, 안전, 유용성, 강력함
일반적인 테마: 기술협력, 연구 및 개발


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune 

In [ ]:
# zero-shot

system_message='Extract keywords from the below text.'
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


In [ ]:
# few-shot

messages=[
    {"role": "system", "content": 'Extract keywords from the corresponding texts below.'},
    {"role": "user", "content": 'Text 1: Stripe provides APIs that web developers can use to integrate payment processing into their websites and mobile applications.'},
    {"role": "assistant", "content": 'Keywords 1: Stripe, payment processing, APIs, web developers, websites, mobile applications'},
    {"role": "user", "content": 'Text 2: OpenAI has trained cutting-edge language models that are very good at understanding and generating text. Our API provides access to these models and can be used to solve virtually any task that involves processing language.'},
    {"role": "assistant", "content": 'Keywords 2: OpenAI, language models, text processing, API.'},
    {"role": "user", "content": 'Text 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])


Keywords 3: OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful.


# 6. 애매한 표현과 부정확한 설명을 줄입니다

In [ ]:
# 애매하고 부정확한 설명

messages=[
    {"role": "system", "content": 'Write a description for a new product.'},
    {"role": "user", "content": 'This product is a new generation of car seat. The description for this product should be fairly short, a few sentences only, and not too much more.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

Introducing the revolutionary new car seat that offers unparalleled safety and comfort for your child. This seat features advanced impact protection and is designed to fit a wide range of vehicles, making it the perfect choice for families on the go. With its sleek, modern design and easy-to-clean fabrics, this car seat will become your go-to solution for safe and comfortable travel.


In [ ]:
# 명확한 설명

messages=[
    {"role": "system", "content": 'Write a description for a new product.'},
    {"role": "user", "content": 'This product is a new generation of car seat. Use a 3 to 5 sentence paragraph to describe this product.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

Introducing the state-of-the-art car seat! This revolutionary product is designed to prioritize the safety and comfort of your child while they are seated in your vehicle. Made from premium quality materials, this car seat boasts adjustable settings that allow it to grow with your child, making it a long-lasting investment. This seat is engineered to absorb energy in the event of a collision, ensuring maximum protection for your child at all times. Additionally, this car seat is easy to install and can be adjusted to different recline angles depending on the preference of your child.


# 7. 하지 말아야 할 것을 말하지 말고 대신 해야 할 것을 말하십시오.

In [ ]:
# 하지 말아야 할 것에 대한 정의 없을 경우 발생하는 사건

messages=[
    {"role": "system", "content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT.'},
    {"role": "user", "content": 'I can’t log in to my account.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

I'm sorry to hear that. Have you tried resetting your password?


In [ ]:
# 해야 하는 것에 대한 정의를 하는 경우

messages=[
    {"role": "system", "content": 'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq'},
    {"role": "user", "content": 'I can’t log in to my account.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

I'm sorry about that! Let's see if we can figure out the issue. Have you tried resetting your password? Sometimes that can help if you're having trouble logging in. You can find instructions on how to reset your password in our help article here: www.samplewebsite.com/help/faq.


# 8. 코드 생성 - 모델을 특정 패턴으로 시작할 수 있도록 "시작하는 단어"를 사용하세요.

In [ ]:
messages=[
    {"role": "user", "content": '# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

def miles_to_km():
    mile = float(input("Enter the distance in miles: "))
    km = mile * 1.60934
    print(mile, "miles is equal to", km, "kilometers")


In [ ]:
# text-davinchi-003 or GPT-4 모델에서 결과물이 더 잘 나옵니다.

messages=[
    {"role": "user", "content": '# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\n\
                import '},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

# text-davinchi-003 에서 테스트한 결과 입니다.
# response = openai.Completion.create(
#             engine=os.getenv('DEPLOYMENT_NAME_003'),
#             prompt='# Write a simple python function that \n\
#                 # 1. Ask me for a number in mile\n\
#                 # 2. It converts miles to kilometers\n\n\
#                 import ',
#             max_tokens=400,
#         )

# print(response['choices'][0]['text'])

### output ###
# math
                
#                 def miles_to_km(miles):
#                     km = miles * 1.609
#                     return math.floor(km)
                    
#                 miles = float(input('Enter a value in miles:'))
#                 km = miles_to_km(miles)
#                 print(f'{miles} miles in kilometers is {km}km')

 math
                
                def miles_to_km(miles):
                    km = miles * 1.609
                    return math.floor(km)
                    
                miles = float(input('Enter a value in miles:'))
                km = miles_to_km(miles)
                print(f'{miles} miles in kilometers is {km}km')
